In [1]:
!pip install git+https://github.com/TracingThePast/automatic_matching

  Cloning https://github.com/TracingThePast/automatic_matching to /private/var/folders/99/chbgg_5s3tn77pkyp7wp3_bm0000gn/T/pip-req-build-8f4083in
  Running command git clone --filter=blob:none --quiet https://github.com/TracingThePast/automatic_matching /private/var/folders/99/chbgg_5s3tn77pkyp7wp3_bm0000gn/T/pip-req-build-8f4083in
  Resolved https://github.com/TracingThePast/automatic_matching to commit 12818379a4c891d743a0df29f95fa2058d729ef3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
from IPython.display import HTML, display
from automatic_matching import get_matching_score, get_results_as_html, match_date_against_local_date, split_string_values, comparison_html_table_cell

## Ablauf:

Die Felder `forenames`, `surnames`, `birth_place` und `death_place` werden soweit es geht in ihre einzelnen Bestandteile zerlegt, das heißt die Aneinanderreihung von Namen `Anna, Anne Anneliese` wird zerlegt in `['Anna', 'Anne', 'Anneliese']` damit die einzelnen Namen jeweils direkt verglichen werden können. Beim Vergleichen wird dann immer das Wertepaar ausgewählt, welches die geringste Levenshtein Distanz aufweist. Abschließend wird aus dem Satz der Minimaldistanzen aller möglichen Paare der Mittelwert berechnet. 

In [3]:


local_test_data = {
    'forenames': ['Margarethe', 'Margarete'],
    'surnames': ['Glaessner', 'Glässner', 'Loewy'],
    'birth_place': ['München'],
    'birth_date': ['1891-09-09'],
    #'death_place': ['Dachau'],
    'death_date': ['1941-12-11'],
}

external_test_data_sets = [
    {
        'forenames': ['Margarete'],
        'surnames': ['Glaessner', 'Loewy'],
        'birth_place': ['München', 'Bavaria'],
        'birth_date': ['1891-09-09'],
        #'death_place': ['Auschwitz'],
        #'death_date': ['1941-11-11'],
    },
    {
        'forenames': ['Grete'],
        'surnames': ['Abeles', 'Loewy',],
        'birth_place': ['München', 'Bavaria'],
        'birth_date': ['1897-09-09'],
        #'death_place': ['Auschwitz'],
        #'death_date': ['1941-11-11'],
    },
    {
        'forenames': ['Grete'],
        'surnames': ['Glaesner', 'Loewy',],
        'birth_place': ['München', 'Bavaria'],
        'birth_date': ['1891-11-08'],
        'death_place': ['Auschwitz'],
        #'death_date': ['1941-11-11'],
    }
]

results = []
for external_test_data in external_test_data_sets:
    results.append(get_matching_score(local_test_data, external_test_data))

results = sorted(results, key=lambda d: (d['absolute_score'], d['relative_score']))
results.reverse()


output_html = get_results_as_html(results)

display(HTML(output_html))

Score,Surname,Forename,Birth date,Birth place,Death date,Death place
Meets criteria: ✅︎ 79.98 / 80 (99.97 %) 79.98 / 80,"glasner (0), lovi (0) [1.00]glasner (0), lovi (0) [1.00] 24.99 / 25",margarete (0) [1.00]margarete (0) [1.00] 24.98 / 25,1891-09-091891-09-09 20.00 / 20,"munchen (0), bavaria (1.00) [-0.50]munchen (0) [1.00] 10.00 / 10",-1941-12-11 - / 10,-- - / 10
Meets criteria: ❌ 31.71 / 80 (39.64 %) 31.71 / 80,"glasner (0), lovi (0) [1.00]glasner (0), lovi (0) [1.00] 24.99 / 25",grete (0.20) [0.81]margarete (0.20) [0.60] 16.73 / 25,1891-11-081891-09-09 -20.00 / 20,"munchen (0), bavaria (1.00) [-0.50]munchen (0) [1.00] 10.00 / 10",-1941-12-11 - / 10,auschviz {Auschwitz}- - / 10
Meets criteria: ❌ 19.72 / 80 (24.65 %) 19.72 / 80,"abeles (1.00), lovi (0) [-0.50]glasner (1.00), lovi (0) [-0.56] -13.49 / 25",grete (0.20) [0.81]margarete (0.20) [0.60] 16.73 / 25,1897-09-091891-09-09 6.48 / 20,"munchen (0), bavaria (1.00) [-0.50]munchen (0) [1.00] 10.00 / 10",-1941-12-11 - / 10,-- - / 10


In [5]:


local_test_data = {
    'forenames': ['Alex', 'Israel'],
    'surnames': ['abrehamsohn'],
    'birth_place': ['München'],
    'birth_date': ['1923-02-01'],
    'death_place': ['Dachau'],
    'death_date': ['1942-11-01'],
}

external_test_data = {
    'forenames': ['Alexander'],
    'surnames': ['abrahams'],
    'birth_place': ['München', 'Bavaria'],
    'birth_date': ['1922-11-11'],
    'death_place': ['Dachau'],
    'death_date': ['1943-02-02'],
}

result = get_matching_score(local_test_data, external_test_data, {'forenames': ['Israel']})

output_html = get_results_as_html(result)

display(HTML(output_html))

Score,Surname,Forename,Birth date,Birth place,Death date,Death place
Meets criteria: ❌ 23.79 / 100 (23.79 %) 23.79 / 100,abrahams (0.27) [0.66]abrehamson (0.27) [0.63] 16.11 / 25,"alexander (0.25) [0.71]alex (0.25), isral (1.00) [0.71] 17.68 / 25",1922-11-111923-02-01 -20.00 / 20,"munchen (0), bavaria (1.00) [-0.50]munchen (0) [1.00] 10.00 / 10",1943-02-021942-11-01 -10.00 / 10,dachau (0) [1.00]dachau (0) [1.00] 10.00 / 10


In [1]:
from IPython.display import HTML, display
from automatic_matching import get_matching_score, get_results_as_html, match_date_against_local_date, split_string_values, comparison_html_table_cell
local_date = '1921-12-01'
result = '<table><tr>'
DATE_MAX_ABSOLUTE_SCORE = 20
MAX_ITEMS_PER_ROW = 5
counter = 0
for external_date in [
    '1912-01-01', '1921-01-12', '1927-01-01', '1920-12-30', '1927-01-12',
    '1921-02-11', '1912-11-02', '1921-07-07', '1920-12-31', '1921-01-21',
    '1912-10-10', '1921-10-10', '1921-10-01', '1921-01-31', '1921-01-12',
    '1921-11-01', '1912-11-11', '1912-**-**', '1912-01-**', '1912-02-**']:
    if counter > 0 and counter % MAX_ITEMS_PER_ROW == 0:
        result += '</tr><tr>'
    date = match_date_against_local_date([local_date], [external_date])
    date['absolute_score'] = date['score'] * DATE_MAX_ABSOLUTE_SCORE
    result += comparison_html_table_cell(DATE_MAX_ABSOLUTE_SCORE, local_value=date.get('local', ''), external_value=date.get('external', ''), score=date.get('score', None), absolute_score=date.get('absolute_score', None))
    counter += 1
result += '</tr></table>'
display(HTML(result))

1912-01-011921-12-01 -20.00 / 20,1921-01-121921-12-01 17.03 / 20,1927-01-011921-12-01 -20.00 / 20,1920-12-301921-12-01 -20.00 / 20,1927-01-121921-12-01 -3.28 / 20
1921-02-111921-12-01 -10.44 / 20,1912-11-021921-12-01 -20.00 / 20,1921-07-071921-12-01 -20.00 / 20,1920-12-311921-12-01 -10.51 / 20,1921-01-211921-12-01 -20.00 / 20
1912-10-101921-12-01 -20.00 / 20,1921-10-101921-12-01 -10.08 / 20,1921-10-011921-12-01 9.91 / 20,1921-01-311921-12-01 -20.00 / 20,1921-01-121921-12-01 17.03 / 20
1921-11-011921-12-01 9.95 / 20,1912-11-111921-12-01 -20.00 / 20,1912-**-**1921-12-01 8.98 / 20,1912-01-**1921-12-01 -20.00 / 20,1912-02-**1921-12-01 -20.00 / 20


In [4]:


local_test_data = {
    'forenames': ['Hanns'],
    'surnames': ['Cohn'],
    'birth_place': ['Berlin', 'Stadt Berlin'],
    'birth_date': ['1923-11-27'],
    'death_place': ['Auschwitz'],
    'death_date': ['<1945-05-08'],
}

external_test_data_sets = [
    {
        'forenames': ['Hans'],
        'surnames': ['Cohn'],
        'birth_place': ['Berlin'],
        'birth_date': ['1923-11-27'],
    },
    {
        'forenames': ['L', 'Hans'],
        'surnames': ['Cohn',],
        'birth_date': ['1924-02-13'],
    },
    {
        'forenames': ['Hanna'],
        'surnames': ['Kohn',],
        'birth_date': ['1924-05-05'],
    },
    {
        'forenames': ['Hanna'],
        'surnames': ['Kohn',],
        'birth_date': ['1924-**-**'],
        'death_place': [],
    }
]

results = []
for external_test_data in external_test_data_sets:
    results.append(get_matching_score(local_test_data, external_test_data))

results = sorted(results, key=lambda d: (d['absolute_score'], d['relative_score']))
results.reverse()


output_html = get_results_as_html(results)

display(HTML(output_html))

Score,Surname,Forename,Birth date,Birth place,Death date,Death place
Meets criteria: ✅︎ 79.99 / 80 (99.99 %) 79.99 / 80,cohn (0) [1.00]cohn (0) [1.00] 25.00 / 25,hans (0) [1.00]hans (0) [1.00] 24.99 / 25,1923-11-271923-11-27 20.00 / 20,"berlin (0) [1.00]berlin (0), stadt berlin (0.25) [0.87] 10.00 / 10",-<1945-05-08 - / 10,-auschviz {Auschwitz} - / 10
Meets criteria: ❌ 58.78 / 70 (83.96 %) 58.78 / 70,kohn (0.06) [0.98]cohn (0.06) [0.98] 24.52 / 25,hana (0.06) [0.97]hans (0.06) [0.97] 24.31 / 25,1924-**-**1923-11-27 9.95 / 20,"-berlin {Berlin}, stadt berlin {Stadt Berlin} - / 10",-<1945-05-08 - / 10,-auschviz {Auschwitz} - / 10
Meets criteria: ❌ 29.99 / 70 (42.84 %) 29.99 / 70,cohn (0) [1.00]cohn (0) [1.00] 25.00 / 25,hans (0) [1.00]hans (0) [1.00] 24.99 / 25,1924-02-131923-11-27 -20.00 / 20,"-berlin {Berlin}, stadt berlin {Stadt Berlin} - / 10",-<1945-05-08 - / 10,-auschviz {Auschwitz} - / 10
Meets criteria: ❌ 28.83 / 70 (41.18 %) 28.83 / 70,kohn (0.06) [0.98]cohn (0.06) [0.98] 24.52 / 25,hana (0.06) [0.97]hans (0.06) [0.97] 24.31 / 25,1924-05-051923-11-27 -20.00 / 20,"-berlin {Berlin}, stadt berlin {Stadt Berlin} - / 10",-<1945-05-08 - / 10,-auschviz {Auschwitz} - / 10


In [5]:


local_test_data = {
    'forenames': ['Hans'],
    'surnames': ['Cohn'],
    'birth_date': ['1921-01-19'],
    'death_place': ['Siegeburg'],
    'death_date': ['1936-12-12'],
}

external_test_data_sets = [
    {
        'forenames': ['Hans', 'M'],
        'surnames': ['Cohn'],
        'birth_date': ['1920-12-27'],
    },
    {
        'forenames': ['Hans'],
        'surnames': ['Kohn',],
        'birth_date': ['1920-06-15'],
        'birth_place': ['Wien'],
    },
    {
        'forenames': ['Hans'],
        'surnames': ['Cohn',],
        'birth_date': ['1920-12-27'],
    },
    {
        'forenames': ['Hans'],
        'surnames': ['Coen',],
        'birth_date': ['1921-02-02'],
    }
]

results = []
for external_test_data in external_test_data_sets:
    results.append(get_matching_score(local_test_data, external_test_data))

results = sorted(results, key=lambda d: (d['absolute_score'], d['relative_score']))
results.reverse()


output_html = get_results_as_html(results)

display(HTML(output_html))

Score,Surname,Forename,Birth date,Birth place,Death date,Death place
Meets criteria: ❌ 30.00 / 70 (42.86 %) 30.00 / 70,cohn (0) [1.00]cohn (0) [1.00] 25.00 / 25,hans (0) [1.00]hans (0) [1.00] 25.00 / 25,1920-12-271921-01-19 -20.00 / 20,-- - / 10,-1936-12-12 - / 10,-siegeburg {Siegeburg} - / 10
Meets criteria: ❌ 30.00 / 70 (42.86 %) 30.00 / 70,cohn (0) [1.00]cohn (0) [1.00] 25.00 / 25,hans (0) [1.00]hans (0) [1.00] 25.00 / 25,1920-12-271921-01-19 -20.00 / 20,-- - / 10,-1936-12-12 - / 10,-siegeburg {Siegeburg} - / 10
Meets criteria: ❌ 29.52 / 70 (42.17 %) 29.52 / 70,kohn (0.06) [0.98]cohn (0.06) [0.98] 24.52 / 25,hans (0) [1.00]hans (0) [1.00] 25.00 / 25,1920-06-151921-01-19 -20.00 / 20,vien {Wien}- - / 10,-1936-12-12 - / 10,-siegeburg {Siegeburg} - / 10
Meets criteria: ❌ 29.19 / 70 (41.70 %) 29.19 / 70,con (0.08) [0.97]cohn (0.08) [0.97] 24.19 / 25,hans (0) [1.00]hans (0) [1.00] 25.00 / 25,1921-02-021921-01-19 -20.00 / 20,-- - / 10,-1936-12-12 - / 10,-siegeburg {Siegeburg} - / 10


In [6]:


local_test_data = {
    'forenames': ['Hans'],
    'surnames': ['Cohn'],
    'birth_date': ['1903-10-02'],
}

external_test_data_sets = [
    {
        'forenames': ['Hans'],
        'surnames': ['Gohn'],
        'birth_date': ['1903-11-01'],
    },
]

results = []
for external_test_data in external_test_data_sets:
    results.append(get_matching_score(local_test_data, external_test_data))

results = sorted(results, key=lambda d: (d['absolute_score'], d['relative_score']))
results.reverse()


output_html = get_results_as_html(results)

display(HTML(output_html))

Score,Surname,Forename,Birth date,Birth place,Death date,Death place
Meets criteria: ❌ 39.47 / 70 (56.39 %) 39.47 / 70,gohn (0.06) [0.98]cohn (0.06) [0.98] 24.52 / 25,hans (0) [1.00]hans (0) [1.00] 25.00 / 25,1903-11-011903-10-02 -10.05 / 20,-- - / 10,-- - / 10,-- - / 10


In [7]:


local_test_data = {
    'forenames': ['Hans'],
    'surnames': ['Cohn'],
    'birth_date': ['1896-01-07'],
    'birth_place': ['Cottbus', 'Mark', 'Brandenburg']
}

external_test_data_sets = [
    {
        'forenames': ['Hans'],
        'surnames': ['Cohn'],
        'birth_date': ['1896-01-07'],
    },
    {
        'forenames': ['Hans'],
        'surnames': ['Kohn'],
        'birth_date': ['1905-03-30'],
    },
]

results = []
for external_test_data in external_test_data_sets:
    results.append(get_matching_score(local_test_data, external_test_data))

results = sorted(results, key=lambda d: (d['absolute_score'], d['relative_score']))
results.reverse()


output_html = get_results_as_html(results)

display(HTML(output_html))

Score,Surname,Forename,Birth date,Birth place,Death date,Death place
Meets criteria: ❌ 39.52 / 70 (56.46 %) 39.52 / 70,cahn (0.06) [0.98]cohn (0.06) [0.98] 24.52 / 25,hans (0) [1.00]hans (0) [1.00] 25.00 / 25,1905-10-131905-10-22 -10.00 / 20,-gotingen {Göttingen} - / 10,-- - / 10,-- - / 10
Meets criteria: ❌ 29.52 / 70 (42.17 %) 29.52 / 70,kohn (0.06) [0.98]cohn (0.06) [0.98] 24.52 / 25,hans (0) [1.00]hans (0) [1.00] 25.00 / 25,1905-03-301905-10-22 -20.00 / 20,-gotingen {Göttingen} - / 10,-- - / 10,-- - / 10


In [12]:


local_test_data = {
    'forenames': ['Hans'],
    'surnames': ['Cohnova'],
    'birth_date': ['1896-01-07'],
    'birth_place': ['Cottbus', 'Mark', 'Brandenburg']
}

external_test_data_sets = [
    {
        'forenames': ['Hans'],
        'surnames': ['Cohn'],
        'birth_date': ['1896-01-07'],
    },
    {
        'forenames': ['Hans'],
        'surnames': ['Kohn'],
        'birth_date': ['1905-03-30'],
    },
]

results = []
for external_test_data in external_test_data_sets:
    results.append(get_matching_score(local_test_data, external_test_data))

results = sorted(results, key=lambda d: (d['absolute_score'], d['relative_score']))
results.reverse()


output_html = get_results_as_html(results)

display(HTML(output_html))

Score,Surname,Forename,Birth date,Birth place,Death date,Death place
Meets criteria: ✅︎ 69.05 / 70 (98.64 %) 69.05 / 70,cohn (0) [1.00]cohn (0) [0.92] 24.05 / 25,hans (0) [1.00]hans (0) [1.00] 25.00 / 25,1896-01-071896-01-07 20.00 / 20,"-cotbus {Cottbus}, mark {Mark}, brandenburg {Brandenburg} - / 10",-- - / 10,-- - / 10
Meets criteria: ❌ 27.37 / 70 (39.10 %) 27.37 / 70,kohn (0.06) [0.98]cohn (0.06) [0.81] 22.37 / 25,hans (0) [1.00]hans (0) [1.00] 25.00 / 25,1905-03-301896-01-07 -20.00 / 20,"-cotbus {Cottbus}, mark {Mark}, brandenburg {Brandenburg} - / 10",-- - / 10,-- - / 10


In [7]:
import icu
latin_transliterator = icu.Transliterator.createInstance('Any-Latin; Latin-ASCII; Lower();')

latin_transliterator.transliterate("Straßmann Käthe".lower().replace("ä", "ae").replace("ü", "ue").replace("ö", "oe"))

'strassmann kaethe'